In [4]:
# 0. Import

In [1]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [6]:
directory = 'D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/'
font_path = directory + 'NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc('font', family=font_name, size=12)
plt.rcParams["figure.figsize"] = (20, 10)
register_matplotlib_converters()

# 1. Data Load
- read.tar
- magazine.json
- metadata.json
- contents 
    - data0~data6
- user.json
- predict 
    - dev.users
    - test.users


In [7]:
magazine = pd.read_json(directory + 'magazine.json', lines=True)
magazine.shape #(27967, 2)
magazine.head()

,id,magazine_tag_list
0,38842,"[브런치북, 육아일기, 대화법, 들려주고픈이야기]"
1,11540,"[tea, food]"
2,11541,[food]
3,11546,"[브런치북, 일상, 시, 사람]"
4,11544,"[감성에세이, 노래, 음악에세이]"


In [65]:
x = 38842
magazine[magazine['id'] == x]

,id,magazine_tag_list
0,38842,"[브런치북, 육아일기, 대화법, 들려주고픈이야기]"


In [8]:
users = pd.read_json(directory + 'users.json', lines=True)
users.shape # (310758, 3)

(310758, 3)

In [9]:
users.head()

,following_list,id,keyword_list
0,"[@perytail, @brunch]",#901985d8bc4c481805c4a4f911814c4a,[]
1,"[@holidaymemories, @wadiz, @sciforus, @dailydu...",#1fd89e9dcfa64b45020d9eaca54e0eed,[]
2,"[@commerceguy, @sunsutu, @kakao-it, @joohoonja...",#1d94baaea71a831e1f33e1c6bd126ed5,[]
3,"[@amberjeon48, @forsy20, @nemotokki, @hawann, ...",#04641c01892b12dc018b1410e4928c0d,[]
4,"[@dwcha7342, @iammento, @kakao-it, @dkam, @ant...",#65bcaff862aadff877e461f54187ab62,[]


In [80]:
empty_list = []
u = []
cnt = 0;
for x in users['keyword_list']:
    if x != empty_list:
        u.append(True)
        cnt+=1
    else :
        u.append(False)
print(cnt)

11544


In [137]:
users[5:10]

,following_list,id,keyword_list,key_true
5,"[@jumi710, @hana8277, @katarun, @brunch3woz, @...",#1a2b23b6332137193be79d297409befb,[],False
6,"[@gabrieljmh, @megaonic, @cleancode, @simu-loo...",#d07b31cfd62b7097837eee6b8328e077,"[{'cnt': 1, 'keyword': '200일 이벤트'}, {'cnt': 2,...",True
7,"[@potatohands, @ggpodori, @chae-pulib, @roysda...",#a6f7a5ff90a19ec4d583f0db1836844d,[],False
8,"[@rmk011, @unitasbrand, @libraryman, @thewater...",#13b3009a8698e9d5e892534d9dcdac62,[],False
9,"[@megustastu, @hongmilmil, @keeuyo, @21mission...",#ba94e11de7e31b606d7a1c8051dba02e,[],False


In [144]:
k = list(users[6:7].keyword_list)

In [158]:
k[0][0]['cnt']
k[0][0]

{'cnt': 1, 'keyword': '200일 이벤트'}

In [10]:
metadata = pd.read_json(directory + 'metadata.json', lines=True)
metadata.shape # (643104, 9)
metadata.head()

,article_id,display_url,id,keyword_list,magazine_id,reg_ts,sub_title,title,user_id
0,782,https://brunch.co.kr/@bookdb/782,@bookdb_782,"[여행, 호주, 국립공원]",8982,1474944427000,세상 어디에도 없는 호주 Top 10,"사진으로 옮기기에도 아까운, 리치필드 국립공원",@bookdb
1,81,https://brunch.co.kr/@kohwang56/81,@kohwang56_81,"[목련꽃, 아지랑이, 동행]",12081,1463092749000,,[시] 서러운 봄,@kohwang56
2,4,https://brunch.co.kr/@hannahajink/4,@hannahajink_4,[],0,1447997287000,무엇 때문에,무엇을 위해,@hannahajink
3,88,https://brunch.co.kr/@bryceandjuli/88,@bryceandjuli_88,"[감정, 마음, 위로]",16315,1491055161000,,싫다,@bryceandjuli
4,34,https://brunch.co.kr/@mijeongpark/34,@mijeongpark_34,"[유럽여행, 더블린, 아일랜드]",29363,1523292942000,#7. 내 친구의 집은 어디인가,Dubliner#7,@mijeongpark


In [170]:
len(metadata['article_id'].unique())

4217

In [159]:
# Read Files

In [161]:
read_file_lst = glob.glob(directory + 'read/read/*')
exclude_file_lst = ['read.tar']

In [162]:
read_df_lst = []
for f in read_file_lst:
    file_name = os.path.basename(f)
    if file_name in exclude_file_lst:
        print(file_name)
    else:
        df_temp = pd.read_csv(f, header=None, names=['raw'])
        df_temp['dt'] = file_name[:8]
        df_temp['hr'] = file_name[8:10]
        df_temp['user_id'] = df_temp['raw'].str.split(' ').str[0]
        df_temp['article_id'] = df_temp['raw'].str.split(' ').str[1:].str.join(' ').str.strip()
        read_df_lst.append(df_temp)

In [163]:
read = pd.concat(read_df_lst)

In [165]:
read.shape #(3507097, 5)
read.head()

,raw,dt,hr,user_id,article_id
0,#e208be4ffea19b1ceb5cea2e3c4dc32c @kty0613_91,20181001,00,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91
1,#0a3d493f3b2318be80f391eaa00bfd1c @miamiyoung_...,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@miamiyoung_31 @banksalad_49 @rlfrjsdn_95 @rea...
2,#b90d3ee7ed0d7d827aae168e159749f1 @joeunha_4 @...,20181001,00,#b90d3ee7ed0d7d827aae168e159749f1,@joeunha_4 @yoonvi_3
3,#b8b9d09fe2961fd62edc94912bf75a90 @hyejinchoi_...,20181001,00,#b8b9d09fe2961fd62edc94912bf75a90,@hyejinchoi_122 @hyejinchoi_86 @hyejinchoi_42 ...
4,#072f742eda9359cdac03ad080193c11d @doyeonsunim...,20181001,00,#072f742eda9359cdac03ad080193c11d,@doyeonsunim_240 @k52524_297 @bule13_33 @kwong...


In [185]:
def chainer(s):
    return list(chain.from_iterable(s.str.split(' ')))
read_cnt_by_user = read['article_id'].str.split(' ').map(len)
read_raw = pd.DataFrame({'dt': np.repeat(read['dt'], read_cnt_by_user),
                         'hr': np.repeat(read['hr'], read_cnt_by_user),
                         'user_id': np.repeat(read['user_id'], read_cnt_by_user),
                         'article_id': chainer(read['article_id'])})
read_raw.shape #(22110706, 4)

(22110706, 4)

In [167]:
read_raw.head()

,dt,hr,user_id,article_id
0,20181001,00,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91
1,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@miamiyoung_31
1,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@banksalad_49
1,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@rlfrjsdn_95
1,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@readme999_140


In [172]:
user_id_list = ['#e208be4ffea19b1ceb5cea2e3c4dc32c','#0a3d493f3b2318be80f391eaa00bfd1c','#b90d3ee7ed0d7d827aae168e159749f1'] 

In [187]:
str(list(read_raw[0:1].user_id)[0]) == user_id_list[0]

True

In [191]:
len(read_raw['user_id'].unique())

306222

In [195]:
cnt=0
for x in list(read_raw['user_id']):
    if(x == user_id_list[0]):
        cnt+=1
print(cnt)

141


In [168]:
print("전체 데이터 건수:", read_raw.shape)
print("중복 소비를 제외한 데이터 건수:", read_raw[['user_id', 'article_id']].drop_duplicates().shape)
print("Unique 독자 수:", len(read_raw['user_id'].unique()))
print("소비된 Unique 글 수:", len(read_raw['article_id'].unique()))

전체 데이터 건수: (22110706, 4)
중복 소비를 제외한 데이터 건수: (12597878, 2)
Unique 독자 수: 306222
소비된 Unique 글 수: 505841
